In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
#from tensorflow import keras
from  tensorflow.keras import layers
from tensorflow.keras.models import Sequential #, Model

In [7]:
class_dict = {'conceptual-art': 0,
 'contemporary': 1,
 'contemporary-realism': 2,
 'light-and-space': 3,
 'minimalism': 4,
 'neo-pop-art': 5,
 'new-european-painting': 6,
 'photorealism': 7,
 'post-minimalism': 8,
 'street-art': 9,
 'transavantgarde': 10}


key_list = list(class_dict.keys())
val_list = list(class_dict.values())

In [8]:
img_height = 256
img_width = 256
input_shape = (img_height, img_width, 3)


def create_dataset_for_prediction(img_folder):
    img_data_array=[]
    for file in os.listdir(img_folder):
        image_path= os.path.join(img_folder, file)
        image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
        try:
            image=cv2.resize(image, (img_height, img_width),interpolation = cv2.INTER_AREA)
        except:
            continue
        image=np.array(image)
        image = image.astype('float32')
        image /= 255 
        if image.shape != input_shape:
            continue
        img_data_array.append([file,image])
    return img_data_array

In [12]:
path = "/content/drive/MyDrive/ArtGreneCNN_Model/4_FinalDataSet/images"
model_path = "/content/drive/MyDrive/ArtGreneCNN_Model/DeepLearningModel"
model_trained = tf.keras.models.load_model(model_path)

In [10]:
def dataArray_to_Genres(dataArray_list, class__dict):
  key_list = list(class__dict.keys())
  val_list = list(class__dict.values())
  input__X = np.array(dataArray_list, np.float32)
  input__Yhat = model_trained.predict(input__X)
  input_class_list = np.array([key_list[val_list.index(np.argmax(input__Yhat, axis=1)[i])] for i in range(len(input__Yhat))])
  return input_class_list

In [13]:
a = create_dataset_for_prediction(path)

In [14]:
for i in range(len(a)):
  a[i][0] = a[i][0][:-4]

In [15]:
input_prediction = dataArray_to_Genres(np.array([x[1] for x in a], np.float32),class_dict)

78/78 [==============================] - 73s 927ms/step


In [16]:
final_prediction = {}
final_prediction['id'] = []
final_prediction['class'] = []
for i in range(len(a)):
  final_prediction['id'].append(a[i][0])
  final_prediction['class'].append(input_prediction[i])
df = pd.DataFrame(final_prediction)

In [20]:
df.to_csv('/content/drive/MyDrive/ArtGreneCNN_Model/style_classification.csv')

In [19]:
!pwd

/content
